<a href="https://colab.research.google.com/github/Rabbito20/PP-Pascal-to-C-generator/blob/main/NikolaDjurovicRN43_2016_faza2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

## Vežbe 5: Grapher

Grapher je opcioni deo kompajlera koji serijalizuje AST pomoću [jezika za opisivanje grafova](https://en.wikipedia.org/wiki/DOT_(graph_description_language)) i prikazuje ga kao sliku.

![pp-01](https://i.postimg.cc/SNmFQ6X0/pp-01.png)

Importovanje neophodnog modula za enumeraciju klasa tokena.

In [ ]:
from enum import Enum, auto

Klasa **Class** definiše sve moguće klase leksema koje se mogu naći u izvornom kodu.

# Lexer

## Lexer I

In [ ]:
class Class(Enum):
    PLUS = auto()       #   +
    MINUS = auto()      #   -
    STAR = auto()       #   *
    FWDSLASH = auto()   #   /
    PERCENT = auto()    #   %
    DIV = auto()        #   mod
    MOD = auto()        #   div

    OR = auto()
    AND = auto()
    NOT = auto()
    XOR = auto()

    EQ = auto()         #   =
    NEQ = auto()        #   !=
    LT = auto()         #   <
    GT = auto()         #   >
    LTE = auto()        #   <=
    GTE = auto()        #   >=

    LPAREN = auto()     #   (
    RPAREN = auto()     #   )
    LBRACKET = auto()   #   [
    RBRACKET = auto()   #   ]
    LBRACE = auto()     #   {
    RBRACE = auto()     #   }

    ASSIGN = auto()     #   :=
    SEMICOLON = auto()  #   ;
    COMMA = auto()      #   ,
    COLON = auto()      #   :
    DOT = auto()        #   .

    #   Tipovi podataka
    TYPE = auto()
    INT = auto()
    CHAR = auto()
    STRING = auto()
    BOOLEAN = auto()
    REAL = auto()

    #   Petlje
    IF = auto()
    ELSE = auto()
    WHILE = auto()
    FOR = auto()

    #   Repeat_until
    REPEAT = auto()
    UNTIL = auto()

    TO = auto()
    DO = auto()
    THEN = auto()
    DOWNTO = auto() #   ovo mora da se doda, nije jos implementirano

    BREAK = auto()
    CONTINUE = auto()

    #   I definisane i nedefinisane funkcije su ID
    ID = auto()
    EOF = auto()

    #   Pocetak i kraj bloka
    BEGIN = auto()
    END = auto()
    #   var, procedura, funkcija
    VAR = auto()        #   VAR
    PROC = auto()       #   PROCEDURA
    FUNC = auto()       #   FUNKCIJA

    #   Niz
    ARRAY = auto()
    OF = auto()

    EXIT = auto()

Klasa **Token** predstavlja uređeni par (klasa, leksema).

Medota **str** vraća string reprezentaciju tokena koja se koristi u procesu pronalaženja grešaka.

In [ ]:
class Token:
    def __init__(self, class_, lexeme):
        self.class_ = class_
        self.lexeme = lexeme

    def __str__(self):
        return "<{} {}>".format(self.class_, self.lexeme)

Klasa **Lekser** sadrži metode za leksičku analizu izvornog koda.

Metoda **lex** formira niz tokena pozivajući metodu **next_token**.

Metoda **next_token** konstruiše token odgovarajuće klase pozivajući metodu **next_char**.

Metoda **next_char** pomera pokazivač na sledeći karakter.

Metoda **read_keyword** konstruiše token ključne reči pod uslovom da je trenutni karakter slovo.

Metoda **read_string** konstruiše token string literala pod uslovom da je trenutni karakter znak navodnika.

Metoda **read_char** konstruiše token literala karaktera pod uslovom da je trenutni karakter apostrof.

Metoda **read_int** konstruiše token literala celog broja pod uslovom da je trenutni karakter cifra.

Metoda **read_space** ne konstruiše token, ali pomera pokazivač na prvi sledeći karakter koji nije razmak.

Metoda **die** se koristi u slučaju da je lekser pročitao neočekivani karakter.

## Lexer II

In [ ]:
class Lexer:
    def __init__(self, text):
        self.text = text
        self.len = len(text)
        self.pos = -1

    def read_space(self):
        while self.pos + 1 < self.len and self.text[self.pos + 1].isspace():
            self.next_char()

    def read_int(self):
        lexeme = self.text[self.pos]
        while self.pos + 1 < self.len and self.text[self.pos + 1].isdigit():
            lexeme += self.next_char()
        return int(lexeme)

    def read_char(self):
        self.pos += 1
        lexeme = self.text[self.pos]
        self.pos += 1
        return lexeme

    def read_string(self):
        lexeme = ''
        while self.pos + 1 < self.len and self.text[self.pos + 1] != '\'':
            lexeme += self.next_char()
        self.pos += 1
        return lexeme

    def read_keyword(self):
        lexeme = self.text[self.pos]
        while self.pos + 1 < self.len and self.text[self.pos + 1].isalnum():
            lexeme += self.next_char()
        if lexeme == 'if':
            return Token(Class.IF, lexeme)
        elif lexeme == 'then':
            return Token(Class.THEN,lexeme)
        elif lexeme == 'else':
            return Token(Class.ELSE, lexeme)
        elif lexeme == 'while':
            return Token(Class.WHILE, lexeme)
        elif lexeme == 'for':
            return Token(Class.FOR, lexeme)
        elif lexeme == 'repeat':
            return Token(Class.REPEAT,lexeme)
        elif lexeme == 'until':
            return Token(Class.UNTIL,lexeme)
        elif lexeme == 'break':
            return Token(Class.BREAK, lexeme)
        elif lexeme == 'continue':
            return Token(Class.CONTINUE, lexeme)
        elif lexeme == 'integer' or lexeme == 'char' or lexeme == 'void' or lexeme == 'boolean' or lexeme == 'real' or lexeme == 'string' or lexeme == 'array':
            return Token(Class.TYPE, lexeme)
        elif lexeme == 'begin':
            return Token(Class.BEGIN,lexeme)
        elif lexeme == 'end':
            return Token(Class.END,lexeme)
        elif lexeme == 'var':
            return Token(Class.VAR,lexeme)
        elif lexeme == 'procedure':
            return Token(Class.PROC,lexeme)
        elif lexeme == 'function':
            return Token(Class.FUNC,lexeme)
        elif lexeme == 'of':
            return Token(Class.OF,lexeme)
        elif lexeme == 'to':
            return Token(Class.TO,lexeme)
        elif lexeme == 'downto':
            return Token(Class.DOWNTO,lexeme)
        elif lexeme == 'do':
            return Token(Class.DO,lexeme)
        elif lexeme == 'div':
            return Token(Class.DIV,lexeme)
        elif lexeme == 'mod':
            return Token(Class.MOD,lexeme)
        elif  lexeme == 'xor':
            return Token(Class.XOR,lexeme)
        elif lexeme == 'and':
            return Token(Class.AND, lexeme)
        elif lexeme == 'or':
            return Token(Class.OR, lexeme)
        elif lexeme == 'not':
            return Token(Class.NOT, lexeme)
        elif lexeme == 'exit':
            return Token(Class.EXIT, lexeme)

        return Token(Class.ID, lexeme)

    def next_char(self):
        self.pos += 1
        if self.pos >= self.len:
            return None
        return self.text[self.pos]

    def next_token(self):
        self.read_space()
        curr = self.next_char()
        if curr is None:
            return Token(Class.EOF, curr)
        token = None
        if curr.isalpha():
            token = self.read_keyword()
        elif curr.isdigit():
            token = Token(Class.INT, self.read_int())
        elif curr == '\'':
              token = Token(Class.STRING, self.read_string())
        elif curr == '"':
            token = Token(Class.STRING, self.read_string())
        elif curr == '+':
            token = Token(Class.PLUS, curr)
        elif curr == '-':
            token = Token(Class.MINUS, curr)
        elif curr == '*':
            token = Token(Class.STAR, curr)
        elif curr == '/':
            token = Token(Class.FWDSLASH, curr)
        elif curr == '=':
              token = Token(Class.EQ, '=')
        elif curr == '<':
            curr = self.next_char()
            if curr == '=':
                token = Token(Class.LTE, '<=')
            if curr == '>':
                token = Token(Class.NEQ)
            else:
                token = Token(Class.LT, '<')
                self.pos -= 1
        elif curr == '>':
            curr = self.next_char()
            if curr == '=':
                token = Token(Class.GTE, '>=')
            else:
                token = Token(Class.GT, '>')
                self.pos -= 1
        elif curr == '(':
            token = Token(Class.LPAREN, curr)
        elif curr == ')':
            token = Token(Class.RPAREN, curr)
        elif curr == '[':
            token = Token(Class.LBRACKET, curr)
        elif curr == ']':
            token = Token(Class.RBRACKET, curr)
        elif curr == '{':
            token = Token(Class.LBRACE, curr)
        elif curr == '}':
            token = Token(Class.RBRACE, curr)
        elif curr == ';':
            token = Token(Class.SEMICOLON, curr)
        elif curr == ',':
            token = Token(Class.COMMA, curr)
        elif curr == ':':
            token = Token(Class.COLON,curr)
            curr = self.next_char()
            if(curr == '='):
              token = Token(Class.ASSIGN,':=')
            else:
                self.pos -= 1
        elif curr == '.':
            token = Token(Class.DOT,curr)

        else:
            self.die(curr)
        return token

    def lex(self):
        tokens = []
        while True:
            curr = self.next_token()
            tokens.append(curr)
            if curr.class_ == Class.EOF:
                break
        return tokens

    def die(self, char):
        raise SystemExit("Unexpected character: {}".format(char))

Klasa **Node** predstavlja baznu klasu za formiranje AST, a klase koje je nasleđuju odgovaraju svakoj ispravnoj semantičkoj strukturi.

# Parser

## Parser I

In [ ]:
class Node():
    pass


class Program(Node):
    def __init__(self, nodes):
        self.nodes = nodes


class Decl(Node):
    def __init__(self, type_, id_):
        self.type_ = type_
        self.id_ = id_


class ArrayDecl(Node):
    def __init__(self, type_, id_, size, elems):
        self.type_ = type_
        self.id_ = id_
        self.size = size
        self.elems = elems


class ArrayElem(Node):
    def __init__(self, id_, index):
        self.id_ = id_
        self.index = index


class Assign(Node):
    def __init__(self, id_, expr):
        self.id_ = id_
        self.expr = expr


class If(Node):
    def __init__(self, cond, true, false):
        self.cond = cond
        self.true = true
        self.false = false


class While(Node):
    def __init__(self, cond, block):
        self.cond = cond
        self.block = block


class For(Node):
    def __init__(self, init, cond, block):
        self.init = init
        self.cond = cond
        self.block = block


class FuncImpl(Node):
    def __init__(self, type_, id_, params, block):
        self.type_ = type_
        self.id_ = id_
        self.params = params
        self.block = block

class ProcImpl(Node):
    def __init__(self, id_, params, block):
        self.id_ = id_
        self.params = params
        self.block = block

class FuncCall(Node):
    def __init__(self, id_, args):
        self.id_ = id_
        self.args = args


class Block(Node):
    def __init__(self, nodes):
        self.nodes = nodes

class Params(Node):
    def __init__(self, params):
        self.params = params


class Args(Node):
    def __init__(self, args):
        self.args = args


class Elems(Node):
    def __init__(self, elems):
        self.elems = elems


class Break(Node):
    pass

class Continue(Node):
    pass

#   Za exit
class ExitCall(Node):
    def __init__(self, bool_):
        #self.bool_ = bool_
        self.bool_ = None

class Return(Node):
    def __init__(self, expr):
        self.expr = expr


class Type(Node):
    def __init__(self, value):
        self.value = value


class Int(Node):
    def __init__(self, value):
        self.value = value


class Char(Node):
    def __init__(self, value):
        self.value = value


class String(Node):
    def __init__(self, value):
        self.value = value


class Id(Node):
    def __init__(self, value):
        self.value = value


class BinOp(Node):
    def __init__(self, symbol, first, second):
        self.symbol = symbol
        self.first = first
        self.second = second


class UnOp(Node):
    def __init__(self, symbol, first):
        self.symbol = symbol
        self.first = first

Klasa **Visitor** predstavlja baznu klasu za obilazak AST.

Metoda **visit** u trenutnom objektu traži metodu koja odgovara tipu prosleđenog čvora.

Metoda **die** se koristi u slučaju da tražena metoda ne postoji, tj. u slučaju kada je potrebno obići čvor čiji tip nije podržan.

In [ ]:
class Visitor():
    def visit(self, parent, node):
        method = 'visit_' + type(node).__name__
        visitor = getattr(self, method, self.die)
        return visitor(parent, node)

    def die(self, parent, node):
        method = 'visit_' + type(node).__name__
        raise SystemExit("Missing method: {}".format(method))

Importovanje neophodnih modula za čuvanje unutrašnjeg stanja objekta.

In [ ]:
from functools import wraps
import pickle

Klasa **Parser** sadrži metode za semantičku analizu izvornog koda koje će iz prosleđenog [FIFO niza](https://en.wikipedia.org/wiki/FIFO_(computing_and_electronics)) tokena formirati AST čvor po čvor.

Metoda **parse** formira AST pomoću [Visitor dizajn šablona](https://sourcemaking.com/design_patterns/visitor) pozivom metode **program**.

Metoda **program** konstruiše AST čvor za deklaraciju globalnih promenljivih i implementaciju funkcija.

Metoda **id_** konstruiše AST čvor za identifikator.

Metoda **decl** konstruiše AST čvor za deklaraciju skalarne promenljive, niza ili funkcije.

Metoda **if_** konstruiše AST čvor za ispitivanje uslova, blok koji se izvršava u slučaju da je uslov tačan i opcioni blok koji se izvršava u slučaju da uslov nije tačan.

Metoda **while_** konstruiše AST čvor za ispitivanje uslova i blok koji se izvršava sve dok je uslov tačan.

Metoda **for_** konstruiše AST čvor za inicijalizaciju iteratora, ispitivanje uslova, inkrementiranje iteratora i blok koji se izvršava sve dok je uslov tačan.

Metoda **block** konstruiše AST čvor za blok instrukcija koje se izvršavaju u okviru neke semantičke celine.

Metoda **params** konstruiše AST čvor za deklarisane parametre funkcije. Svaki parametar ima naziv i tip.

Metoda **args** konstruiše AST čvor za prosleđene argumente pozivu funkcije. Svaki argument ima naziv i vrednost.

Metoda **elems** konstruiše AST čvor za definisane elemente pri inicijalizaciji niza.

Metoda **return_** konstruiše AST čvor za prekid funkcije uz opciono vraćanje vrednosti.

Metoda **break_** konstruiše AST čvor za prekid petlje.

Metoda **continue_** konstruiše AST čvor za skok na sledeću iteraciju petlje.

Metoda **type_** konstruiše AST čvor za tip podataka, tj. "int", "char" ili "void".

Metoda **factor** konstruiše AST čvor za matematičke operacije visokog prioriteta, tj. unarne operacije.

Metoda **term** konstruiše AST čvor za matematičke operacije srednjeg prioriteta, tj. multiplikativne operacije.

Metoda **expr** konstruiše AST čvor za matematičke operacije niskog prioriteta, tj. aditivne operacija.

Metoda **compare** konstruiše AST čvor za poređenje dva logička operanda.

Metoda **logic** konstruiše AST čvor za logičku konjunkciju i disjunkciju.

Metoda **eat** uzima token za početka niza i proverava da li njegova klasa odgovara prosleđenoj klasi.

Metoda **is_func_call** proverava da li trenutni identifikator odgovara pozivu ili implementaciji funkcije. Nakon provere vraća parser u originalno stanje.

Metoda **restorable** se dodaje kao anotacija drugoj metodi koja menja unutrašnje stanje objekta, a potrebno je da se objekat po završetku funkcije vrati u originalno stanje.

Metoda **die** se koristi u slučaju da se dogodi bilo koja greška.

Metoda **die_deriv** se koristi u slučaju da pročitani token ne odgovara sementičkoj strukturi koja se trenutno formira.

Metoda **die_type** se koristi u slučaju da klasa tokena sa početka niza ne odgovara klasi prosleđenoj pozivu metode **eat**.

## Parser II

In [ ]:
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.curr = tokens.pop(0)
        self.prev = None

    # Vraca stanje nekog objekta True ili False (ovo je sablon)
    def restorable(call):
        @wraps(call)
        def wrapper(self, *args, **kwargs):
            state = pickle.dumps(self.__dict__)
            result = call(self, *args, **kwargs)
            self.__dict__ = pickle.loads(state)
            return result
        return wrapper

    def eat(self, class_):
        if self.curr.class_ == class_:
            self.prev = self.curr
            self.curr = self.tokens.pop(0)
        else:
            self.die_type(class_.name, self.curr.class_.name)

    def program(self):
        nodes = []
        while self.curr.class_ != Class.EOF:
            #print('\nprogram start ->', self.curr.lexeme, '-', self.curr.class_)        #DEBUG
            if self.curr.class_ == Class.PROC:
                nodes.append(self.proc())
                continue
            if self.curr.class_ == Class.FUNC:
                nodes.append(self.func())
                continue
            if self.curr.class_ == Class.BEGIN:
                nodes.append(self.block())
            if self.curr.class_ == Class.VAR:
                self.eat(Class.VAR)
                while self.curr.class_ not in [Class.BEGIN, Class.FUNC, Class.PROC]:
                  cvorovi = self.decl()
                  for c in cvorovi:
                    nodes.append(c)
                continue
            if self.curr.class_ == Class.DOT:
                self.eat(Class.DOT)
                if self.curr.class_ != Class.EOF:
                    self.die_deriv(self.program.__name__)
            else:
                self.die_deriv(self.program.__name__)
        return Program(nodes)

    # Metoda id_ konstruiše AST čvor za identifikator.
    # Ovo bi trebalo da radi i za procedure i funkcije
    def id_(self):
        id_ = Id(self.curr.lexeme)
        #print('     |   id ->', self.curr.lexeme, '-', self.curr.class_)        #DEBUG
        self.eat(Class.ID)
        if self.curr.class_ == Class.LPAREN: # and self.is_func_call():
            self.eat(Class.LPAREN)
            args = self.args()
            self.eat(Class.RPAREN)
            return FuncCall(id_, args)
        elif self.curr.class_ == Class.LBRACKET:
            self.eat(Class.LBRACKET)
            index = self.expr()
            self.eat(Class.RBRACKET)
            id_ = ArrayElem(id_, index)
        if self.curr.class_ == Class.ASSIGN:
            self.eat(Class.ASSIGN)
            expr = self.logic()
            return Assign(id_, expr)
        else:
            return id_

    def func_id_(self):
        id_ = Id(self.curr.lexeme)
        #print(' func_id ->', self.curr.lexeme, '-', self.curr.class_)           #DEBUG
        self.eat(Class.ID)
        return id_

    # Metoda decl konstruiše AST čvor za deklaraciju skalarne promenljive,
    # niza ili funkcije.
    def decl(self):
        var_nodes = []
        var_nodes.append(self.id_())

        while self.curr.class_ == Class.COMMA:
          self.eat(Class.COMMA)
          var_nodes.append(self.id_())

        self.eat(Class.COLON)
        #print('     |-> decl type   ->', self.curr.lexeme, '-', self.curr.class_)        #DEBUG
        type_ = self.type_()

        if type_.value == 'string':
          if self.curr.class_ == Class.LBRACKET:
            self.eat(Class.LBRACKET)
            size = None
            if self.curr.class_ != Class.RBRACKET:
              size = self.expr()
            self.eat(Class.RBRACKET)
            elems = None
          # elem TODO
            ar_declarations = [
              ArrayDecl(type_, id_node, size, elems)
              for id_node in var_nodes ]
            self.eat(Class.SEMICOLON)
            return ar_declarations
        elif type_.value == 'array':
          #print("neki niz")
          self.eat(Class.LBRACKET)
          min_index = self.factor()
          self.eat(Class.DOT)
          self.eat(Class.DOT)
          max_index = self.factor()
          self.eat(Class.RBRACKET)
          self.eat(Class.OF)
          type_ = self.type_()
          elems = None
          if self.curr.class_ == Class.EQ:
            self.eat(Class.EQ)
            self.eat(Class.LPAREN)
            elems = self.args()
            self.eat(Class.RPAREN)
          self.eat(Class.SEMICOLON)
          arrs_decs = [ArrayDecl(type_, var_nodes[0], max_index, elems)]
          return arrs_decs


        self.eat(Class.SEMICOLON)
        var_declarations = [
              Decl(type_,id_node)
              for id_node in var_nodes
        ]
        return var_declarations

    def func(self):
        self.eat(Class.FUNC)
        #print('     func ->', self.curr.lexeme)                                 #DEBUG
        #print(' func ->', self.curr.lexeme)                                     #DEBUG
        id_ = self.func_id_()
        self.eat(Class.LPAREN)
        params = self.params()
        self.eat(Class.RPAREN)
        self.eat(Class.COLON)
        type_ = self.type_()
        self.eat(Class.SEMICOLON)
        block = self.block()
        self.eat(Class.SEMICOLON)
        return FuncImpl(type_, id_, params, block)

    def proc(self):
        self.eat(Class.PROC)
        #print(' proc ->', self.curr.lexeme)                                     #DEBUG
        #print(' proc ->', self.curr.class_)                                     #DEBUG
        id_ = self.func_id_()
        self.eat(Class.LPAREN)
        params = self.params()
        self.eat(Class.RPAREN)
        self.eat(Class.SEMICOLON)
        nodes = []
        if self.curr.class_ == Class.VAR:
            self.eat(Class.VAR)
            while self.curr.class_ not in [Class.BEGIN, Class.FUNC, Class.PROC]:
                cvorovi = self.decl()
                for p in cvorovi:
                  nodes.append(p)
        block = self.block()
        self.eat(Class.SEMICOLON)
        return ProcImpl(id_, params, block)


    def if_(self):
        self.eat(Class.IF)
        cond = self.logic()
        self.eat(Class.THEN)
        true = self.block()
        #   Ovaj if se aktivira u slucaju kad postoji else
        #if self.curr.lexeme == Class.SEMICOLON:
            #self.eat(Class.SEMICOLON)          #hmmmmm
        false = None
        if self.curr.class_ == Class.ELSE:
            self.eat(Class.ELSE)
            false = self.block()
        self.eat(Class.SEMICOLON)
        return If(cond, true, false)

    def while_(self):
        self.eat(Class.WHILE)
        cond = self.logic()
        self.eat(Class.DO)
        block = self.block()
        self.eat(Class.SEMICOLON)
        return While(cond, block)

    def for_(self):
        self.eat(Class.FOR)
        init = self.id_()
        self.eat(Class.TO)
        cond = self.expr()
        self.eat(Class.DO)
        bloc = self.block()
        self.eat(Class.SEMICOLON)
        return For(init, cond, bloc)

    def exit_(self):
        pom = []
        self.eat(Class.EXIT)
        false = None    #   diskutabilno
        if self.curr.class_ == Class.LPAREN:
            self.eat(Class.LPAREN)
            #pom.append(self.args())
            false = self.id_()
            self.eat(Class.RPAREN)
        #print('exit_ ---------------------------------->', false)              #DEBUG
        pom.append(false)
        #Ne radi!!!! Dovrsi!
        #   Radi parser, ali ima problem sa generisanjem stabla
        self.eat(Class.SEMICOLON)
        return ExitCall(pom)

    def block(self):
        nodes = []
        self.eat(Class.BEGIN)
        while self.curr.class_ != Class.END:
            #print('     block ->', self.curr.lexeme, '-', self.curr.class_)     #DEBUG
            if self.curr.class_ == Class.IF:
                nodes.append(self.if_())
            elif self.curr.class_ == Class.WHILE:
                nodes.append(self.while_())
            elif self.curr.class_ == Class.FOR:
                nodes.append(self.for_())
            elif self.curr.class_ == Class.BREAK:
                nodes.append(self.break_())
            elif self.curr.class_ == Class.CONTINUE:
                nodes.append(self.continue_())
            elif self.curr.class_ == Class.TYPE:
                nodes.append(self.decl())
            elif self.curr.class_ == Class.ID:
                nodes.append(self.id_())
                self.eat(Class.SEMICOLON)
            elif self.curr.class_ == Class.EXIT:
                #pom = self.curr.lexeme
                #print('ULAZIMO U EXIT ???????????????????????????')             #DEBUG
                nodes.append(self.exit_())
                #self.eat(Class.SEMICOLON)  #   Jedemo u exit_()
            else:
                #print('     JEDEMO block err ->', self.curr.lexeme, '-', self.curr.class_)        #DEBUG
                #self.eat(self.curr.class_)                                      #DEBUG
                #continue
                self.die_deriv(self.block.__name__)
        self.eat(Class.END)
        #   Mozda i ne treba ovo
        #if self.curr.class_ == Class.SEMICOLON:
            #self.eat(Class.SEMICOLON)
        return Block(nodes)

    def params(self):
        params = []
        type_ = None
        while self.curr.class_ != Class.RPAREN:
            if len(params) > 0:
                self.eat(Class.COMMA)
            id_ = self.id_()
            params.append(id_)
            if self.curr.class_ == Class.COLON:
              self.eat(Class.COLON)
              type_ = self.type_()
        prs = [ Decl(type_, some_id) for some_id in params]
        return Params(prs)

    def args(self):
        args = []
        while self.curr.class_ != Class.RPAREN:
            #print('         |   args ->', self.curr.lexeme)                     #DEBUG
            if len(args) > 0:
                self.eat(Class.COMMA)
            args.append(self.expr())
        return Args(args)

    def elems(self):
        elems = []
        while self.curr.class_ != Class.RBRACE:
            #print('         elems ->', self.curr.lexeme)                        #DEBUG
            if len(elems) > 0:
                self.eat(Class.COMMA)
            elems.append(self.expr())
        return Elems(elems)

    def break_(self):
        self.eat(Class.BREAK)
        self.eat(Class.SEMICOLON)
        return Break()

    def continue_(self):
        self.eat(Class.CONTINUE)
        self.eat(Class.SEMICOLON)
        return Continue()

    def type_(self):
        type_ = Type(self.curr.lexeme)
        self.eat(Class.TYPE)
        return type_

    def factor(self):
        if self.curr.class_ == Class.INT:
            value = Int(self.curr.lexeme)
            self.eat(Class.INT)
            return value
        elif self.curr.class_ == Class.CHAR:
            value = Char(self.curr.lexeme)
            self.eat(Class.CHAR)
            return value
        elif self.curr.class_ == Class.STRING:
            value = String(self.curr.lexeme)
            self.eat(Class.STRING)
            return value
        elif self.curr.class_ == Class.ID:
            return self.id_()
        elif self.curr.class_ in [Class.MINUS, Class.NOT]:
            op = self.curr
            self.eat(self.curr.class_)
            first = None
            if self.curr.class_ == Class.LPAREN:
                self.eat(Class.LPAREN)
                first = self.logic()
                self.eat(Class.RPAREN)
            else:
                first = self.factor()
            return UnOp(op.lexeme, first)
        elif self.curr.class_ == Class.LPAREN:
            self.eat(Class.LPAREN)
            first = self.expr()
            self.eat(Class.RPAREN)
            return first
        elif self.curr.class_ == Class.SEMICOLON:
            return None
        else:
            self.die_deriv(self.factor.__name__)

    def term(self):
        first = self.factor()
        while self.curr.class_ in [Class.STAR, Class.FWDSLASH, Class.PERCENT, Class.MOD, Class.DIV]:
            #print('         term ->', self.curr.lexeme)                         #DEBUG
            if self.curr.class_ == Class.STAR:
                op = self.curr.lexeme
                self.eat(Class.STAR)
                second = self.factor()
                first = BinOp(op, first, second)
            elif self.curr.class_ == Class.FWDSLASH:
                op = self.curr.lexeme
                self.eat(Class.FWDSLASH)
                second = self.factor()
                first = BinOp(op, first, second)
            elif self.curr.class_ == Class.PERCENT:
                op = self.curr.lexeme
                self.eat(Class.PERCENT)
                second = self.factor()
                first = BinOp(op, first, second)
            elif self.curr.class_ == Class.MOD:
                op = self.curr.lexeme
                self.eat(Class.MOD)
                second = self.factor()
                first = BinOp(op, first, second)
            elif self.curr.class_ == Class.DIV:
                op = self.curr.lexeme
                self.eat(Class.DIV)
                second = self.factor()
                first = BinOp(op, first, second)
        return first

    def expr(self):
        first = self.term()
        while self.curr.class_ in [Class.PLUS, Class.MINUS]:
            #print('         expr ->', self.curr.lexeme)                         #DEBUG
            if self.curr.class_ == Class.PLUS:
                op = self.curr.lexeme
                self.eat(Class.PLUS)
                second = self.term()
                first = BinOp(op, first, second)
            elif self.curr.class_ == Class.MINUS:
                op = self.curr.lexeme
                self.eat(Class.MINUS)
                second = self.term()
                first = BinOp(op, first, second)
        return first

    def compare(self):
        first = self.expr()
        if self.curr.class_ == Class.EQ:
            op = self.curr.lexeme
            self.eat(Class.EQ)
            second = self.expr()
            return BinOp(op, first, second)
        elif self.curr.class_ == Class.NEQ:
            op = self.curr.lexeme
            self.eat(Class.NEQ)
            second = self.expr()
            return BinOp(op, first, second)
        elif self.curr.class_ == Class.LT:
            op = self.curr.lexeme
            self.eat(Class.LT)
            second = self.expr()
            return BinOp(op, first, second)
        elif self.curr.class_ == Class.GT:
            op = self.curr.lexeme
            self.eat(Class.GT)
            second = self.expr()
            return BinOp(op, first, second)
        elif self.curr.class_ == Class.LTE:
            op = self.curr.lexeme
            self.eat(Class.LTE)
            second = self.expr()
            return BinOp(op, first, second)
        elif self.curr.class_ == Class.GTE:
            op = self.curr.lexeme
            self.eat(Class.GTE)
            second = self.expr()
            return BinOp(op, first, second)
        else:
            return first

    def logic_term(self):
        first = self.compare()
        while self.curr.class_ == Class.AND:
            op = self.curr.lexeme
            self.eat(Class.AND)
            second = self.compare()
            first = BinOp(op, first, second)
        while self.curr.class_ == Class.XOR:
            op = self.curr.lexeme
            self.eat(Class.XOR)
            second = self.compare()
            first = BinOp(op, first, second)
        return first

    def logic(self):
        first = self.logic_term()
        while self.curr.class_ == Class.OR:
            op = self.curr.lexeme
            self.eat(Class.OR)
            second = self.logic_term()
            first = BinOp(op, first, second)
        return first

    @restorable
    def is_func_call(self):
        try:
            self.eat(Class.LPAREN)
            self.params()
            self.eat(Class.RPAREN)
            return self.curr.class_ != Class.LBRACE
        except:
            return False

    def parse(self):
        return self.program()

    def die(self, text):
        raise SystemExit(text)

    def die_deriv(self, fun):
        self.die("Derivation error: {}".format(fun))

    def die_type(self, expected, found):
        self.die("Expected: {}, Found: {}".format(expected, found))

# Generator

In [ ]:
import re

Klasa **Generator** sadrži metode za generisanje koda na osnovu AST.

Metoda **generate** generiše kod rekurzivnim pozivom **visit** metode. Svaki čvor AST ima svoju string reprezentaciju i ona se dodaje na kraj stringa koji će biti upisan u fajl.

Metoda **append** dodaje prosleđeni string na dosadašnji generisani kod.

Metoda **newline** dodaje karaktere za prelazak u novu liniju na dosadašnji generisani kod.

Metoda **indent** dodaje tabulatore za prelazak u novi blok instrukcija na dosadašnji generisani kod.

In [ ]:
class Generator(Visitor):
    def __init__(self, ast):
        self.ast = ast
        self.c = ""
        self.level = 0

    #   Metoda append dodaje prosleđeni string na dosadašnji generisani kod
    def append(self, text):
        self.c += str(text)

    #   Novi red
    def newline(self):
        self.append('\n\r')

    #   Nije nam neophodna, ali cini kod citljivijim
    #   Dodaje nam tabove
    def indent(self):
        for i in range(self.level):
            self.append('\t')

    #   Moj
    def visit_Program(self, parent, node):
        self.append('#include<stdio.h>')
        self.newline()
        self.append('void main()')
        self.newline()
        #self.level += 1
        self.indent()
        #   Prolazimo kroz cvorove unutar koda
        for n in node.nodes:
            self.visit(node, n)

        self.newline()
        #self.append('}')       #   Posto smo dodali u visit_block
        self.newline()
        #self.level -= 1

    """
    #   Vezbe
    def visit_Program(self, parent, node):
        for n in node.nodes:
            self.visit(node, n)
        self.append('if __name__ == "__main__":')
        self.newline()
        self.level += 1
        self.indent()
        self.append('main()')
        self.newline()
        self.level -= 1
    """

    #   Hmmmmmm
    def visit_Decl(self, parent, node):
        #self.visit(node, node.type_)
        #for n in node.nodes:
            #self.visit(node, n)
        pass


    def visit_ArrayDecl(self, parent, node):
        self.visit(node, node.id_)
        if node.elems is not None:
            self.append(' = [')
            self.visit(node, node.elems)
            self.append(']')
        elif node.size is not None:
            self.append(' = ')
            self.visit(node, node.size)
            self.append(' * [None]')

    #   Ovo bi trebalo da bude isto
    def visit_ArrayElem(self, parent, node):
        self.visit(node, node.id_)
        self.append('[')
        self.visit(node, node.index)
        self.append(']')

    #   Ostaje isto
    def visit_Assign(self, parent, node):
        self.visit(node, node.id_)
        self.append(' = ')
        self.visit(node, node.expr)

    def visit_If(self, parent, node):
        self.append('if ')
        self.append('(')
        self.visit(node, node.cond)
        self.append(')')
        self.newline()
        #self.indent()
        self.visit(node, node.true)
        #self.indent()
        if node.false is not None:
            self.newline()
            self.indent()
            self.append('else')
            self.newline()
            self.visit(node, node.false)

    def visit_While(self, parent, node):
        self.append('while (')
        self.visit(node, node.cond)
        self.append(')')
        self.newline()
        self.visit(node, node.block)

    def visit_For(self, parent, node):
        self.visit(node, node.init)
        self.newline()
        self.indent()
        #   U sustini for prelazi u while
        self.append('while (')
        self.visit(node, node.cond)
        self.append(')')
        self.newline()
        self.visit(node, node.block)
        self.level += 1
        self.indent()
        self.visit(node, node.step)
        self.level -= 1

    def visit_FuncImpl(self, parent, node):
        #   dodaj tip f-je
        #self.append(node.id_.visit_Type)    # Mislim da je ovako hmmmmm...
        self.append('\n')
        self.append(node.id_.value)    #   Isto radi kao i linija ispod
        #self.append(node.id_)
        self.append('(')
        self.visit(node, node.params)
        self.append(')')
        self.newline()
        self.visit(node, node.block)

    def visit_ExitCall(self, parent, node):
        pass

    #   Posle vrati isto za funkciju
    def visit_ProcImpl(self, parent, node):
        #   Procedura je void funkcija
        #self.append(node.id_.visit_Type)
        self.append('void ')
        self.append(node.id_.value)
        self.append('(')
        self.visit(node, node.params)
        self.append(')')
        self.newline()
        self.visit(node, node.block)

    #   Dovrsi
    def visit_FuncCall(self, parent, node):
        #pass
        func = node.id_.value   #   Ime funkcije
        args = node.args.args   #   Argumenti i njihove vrednosti (Niz argumenata)
        #####################################
        #   readln -> scanf()
        #print('func call -- func  ->', func)                                            #DEBUG
        if func == 'readln':
            #print('U -- readln')                                                #DEBUG
            self.append('scanf("')
            if len(args) > 1:
                for a in args:
                    #print('func call -- args ->', a)                                            #DEBUG
                    if isinstance(a, Char):
                        self.append('%c')
                    elif isinstance(a, Int):
                        self.append('%d')
                    elif isinstance(a, String):
                        self.append('%s')
                    else:
                        self.append(a.node.id_)
            #for i, a in enumerate(args[1:]):
                #print('u FORu func -> scanf ->', a)                                    #DEBUG
                #   Ne treba vrv
                #if i > 0:
                    #self.append(' ')
                self.visit(node.args, a)
            self.append('"')
            self.append(',')
            if len(args) > 1:
                for t in range(args):
                    #print('func call -- args2 ->', t)                                            #DEBUG
                    #if isinstance(a, Char):
                    self.append(t.id_)
                    self.append(t.args)
                    if t == len(args):
                        self.append(');')
                    else:
                        self.append(',')

            else:
                self.append(');')


            #self.append('%d')
            #if len(args[1:]) > 1:
                #self.append('.split()')
            #format_ = args[0].value
            #matches = re.findall('%[dcs]', format_)
            '''
            for i, m in enumerate(matches):
                if m == '%d':
                    self.newline()
                    self.indent()
                    self.visit(node.args, args[i + 1])
                    self.append(' = int(')
                    self.visit(node.args, args[i + 1])
                    self.append(')')
                elif m == '%c':
                    self.newline()
                    self.indent()
                    self.visit(node.args, args[i + 1])
                    self.append(' = ord(')
                    self.visit(node.args, args[i + 1])
                    self.append('[0])')
                elif m == '%s':
                    self.newline()
                    self.indent()
                    self.visit(node.args, args[i + 1])
                    self.append(' = [ord(x) for x in ')
                    self.visit(node.args, args[i + 1])
                    self.append(']')
            self.append(');')
            '''


        #   write -> printf()
        elif func == 'write':
            self.append('printf("')
            #format_ = args[0].value

            #matches = re.findall('%[dcs]', format_)    #ne

            #format_ = re.sub('%[dcs]', '{}', format_)

            #self.append(format_)
            self.append('"')
            #if len(args) > 1:

            #   printf()
            self.newline()

            #   Strlen, strcat - hmmmm

        #   Opsti slucaj
        else:
            self.append(func)
            self.append('(')
            self.visit(node, node.args)
            self.append(')')
            #self.append(';')    #   Nisam siguran za ovo
        #####################################

    #   Za c->pascal nam ne treba
    #   Zbog estetike cuvamo
    def visit_Block(self, parent, node):
        self.level += 1
        self.indent()
        self.append('{')
        self.newline()
        for n in node.nodes:
            self.indent()
            #self.newline()
            self.visit(node, n)
            self.newline()
            self.indent()
        #self.indent()
        self.append('}')
        self.level -= 1

    #   Parametri, elementi i argumenti se isto implementiraju
    def visit_Params(self, parent, node):
        for i, p in enumerate(node.params):
            if i > 0:
                self.append(', ')
            self.visit(p, p.id_)

    def visit_Args(self, parent, node):
        for i, a in enumerate(node.args):
            if i > 0:
                self.append(', ')
            self.visit(node, a)

    def visit_Elems(self, parent, node):
        for i, e in enumerate(node.elems):
            if i > 0:
                self.append(', ')
            self.visit(node, e)

    #   break i continue su isti, samo smo dodali ';'
    def visit_Break(self, parent, node):
        self.append('break;')

    def visit_Continue(self, parent, node):
        self.append('continue;')

    #   Ovo nam ne treba, pascal nema return
    '''
    def visit_Return(self, parent, node):
        self.append('return')
        if node.expr is not None:
            self.append(' ')
            self.visit(node, node.expr)
    '''

    #   Vracamo tip
    def visit_Type(self, parent, node):
        self.visit(node, node.first)
        #int boolean char string
        #if node

    def visit_Int(self, parent, node):
        self.append(node.value)

    def visit_Char(self, parent, node):
        self.append(ord(node.value))

    def visit_String(self, parent, node):
        self.append(node.value)

    #   Valjda se ne menja
    def visit_Id(self, parent, node):
        self.append(node.value)

    def visit_BinOp(self, parent, node):
        self.visit(node, node.first)
        if node.symbol == 'and':
            self.append(' && ')
        elif node.symbol == 'or':
            self.append(' || ')
        elif node.symbol == 'xor':
            self.append('^')
        elif node.symbol == '/' or node.symbol == 'div':
            self.append('/')
        elif node.symbol == 'mod' or node.symbol == '%':
            self.append('%')

        #   Valjda nisu neophodni ostali simboli (+, -, *), jer ce se oni samo prepisati onda

        else:
            self.append(node.symbol)
        self.visit(node, node.second)

    def visit_UnOp(self, parent, node):
        if node.symbol == '!' or node.symbol == 'not':
            self.append('!')
        #elif node.symbol != '&':
            #self.append(node.symbol)
        #elif node.symbol == 'or':
            #self.append('|')   #   Nije unarna operacija!
        self.visit(node, node.first)

    #   Glavna metoda ovde, generise kod
    def generate(self, path):
        self.visit(None, self.ast)
        self.c = re.sub('\n\s*\n', '\n', self.c)  #   Menja beline enterom
        with open(path, 'w') as source:
            source.write(self.c)
        return path

# Test

Testiranje implementacije.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_id = '04'

#   Prva faza
#path = f'/content/drive/MyDrive/Predmeti i beleske/Programski prevodioci/Projekat/Datoteke/Prva faza/pas/test{test_id}.pas'
#   Druga faza
path = f'/content/drive/MyDrive/RAF - Predmeti i beleske/PP/Projekat/Datoteke/Druga faza/{test_id}/src.pas'

with open(path, 'r') as source:
    text = source.read()

    #   Lexer
    lexer = Lexer(text)
    tokens = lexer.lex()

    for t in tokens:
        #print(t)
        pass

    #   Parser i stablo
    parser = Parser(tokens)
    ast = parser.parse()
    #print("\n AST radi\n")

    #   Grapher
    ''' Nije implementiran '''
    # grapher = Grapher(ast)
    # img = grapher.graph()

    #   Generator
    #print('Generisani kod:\n')
    generator = Generator(ast)
    code = generator.generate('main.c')


#   Slika
# Image(img)

#   Kod
!cat '{code}'


#include<stdio.h>
void main()
void checkarm(x, cj, cd, cs)
	{
	if (x<0)
		{
		}
		arm = x=cj*cj*cj+cd*cd*cd+cs*cs*cs
		if (arm)
		{
		printf(""
		}
	else
		{
		printf(""
		}
	}	{
	scanf("",);
		cj = broj%10
		cd = broj/10%10
		cs = broj/100%10
		checkarm(broj, cj, cd, cs)
	}


In [ ]:
# Image(img)
#   prikaz koda

In [ ]:
#!gcc '{code}'